### Process daily UNSEEN ###

Processes the model data for a given initialisation year (e.g. s1960), ensemble member (e.g. r1i1p1f2), model (e.g. HadGEM3-GC31-MM), spatial area (e.g. UK) and variable (e.g. tas) into a dataframe.

In [1]:
# Local imports
import os
import sys
import time
import argparse

# Third-party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import shapely.geometry
import cartopy.io.shapereader as shpreader
import iris

# Specific imports
from tqdm import tqdm
from datetime import datetime, timedelta

/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# Load my specific functions
sys.path.append("/home/users/benhutch/unseen_functions")
import functions as funcs

In [3]:
# Hard coded args
model = "HadGEM3-GC31-MM"
experiment = "dcppA-hindcast"
freq = "day"
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Set up the output directory for the dfs
output_dir_dfs = "/gws/nopw/j04/canari/users/benhutch/unseen/saved_dfs"

In [4]:
# Flexible args
variable = "tas"
country = "United Kingdom"
init_year = 1960

In [5]:
# r10i1p1f2' 'r1i1p1f2' 'r2i1p1f2' 'r3i1p1f2' 'r4i1p1f2' 'r5i1p1f2'
#  'r6i1p1f2' 'r7i1p1f2' 'r8i1p1f2' 'r9i1p1f2'

In [6]:
%%time

# Load the model data
model_ds = funcs.load_model_data_xarray(
    model_variable=variable,
    model=model,
    experiment=experiment,
    start_year=init_year,
    end_year=init_year,
    first_fcst_year=init_year + 1,
    last_fcst_year=init_year + 10,
    months=months,
    member="r1i1p1f2",
    frequency=freq,
    parallel=False,
)

Model path: /badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast
The model path root is badc
Number of unique variant labels: 10
For model: HadGEM3-GC31-MM
First 10 unique variant labels: ['r10i1p1f2' 'r1i1p1f2' 'r2i1p1f2' 'r3i1p1f2' 'r4i1p1f2' 'r5i1p1f2'
 'r6i1p1f2' 'r7i1p1f2' 'r8i1p1f2' 'r9i1p1f2']
Number of unique variant labels: 10
Unique variant labels: ['r10i1p1f2' 'r1i1p1f2' 'r2i1p1f2' 'r3i1p1f2' 'r4i1p1f2' 'r5i1p1f2'
 'r6i1p1f2' 'r7i1p1f2' 'r8i1p1f2' 'r9i1p1f2']
First 10 model files: ['/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r10i1p1f2/day/tas/gn/files/d20200417/tas_day_HadGEM3-GC31-MM_dcppA-hindcast_s1960-r10i1p1f2_gn_19601101-19601230.nc', '/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r10i1p1f2/day/tas/gn/files/d20200417/tas_day_HadGEM3-GC31-MM_dcppA-hindcast_s1960-r10i1p1f2_gn_19610101-19611230.nc', '/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r10i1p1f2/day/tas/gn/files/d20200417/tas_d

Processing init years: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s]

CPU times: user 450 ms, sys: 78.2 ms, total: 529 ms
Wall time: 689 ms


In [7]:
model_ds

<xarray.Dataset> Size: 2GB
Dimensions:    (init: 1, member: 1, lead: 3750, bnds: 2, lat: 324, lon: 432)
Coordinates:
  * lead       (lead) int64 30kB 1 2 3 4 5 6 7 ... 3745 3746 3747 3748 3749 3750
  * lat        (lat) float64 3kB -89.72 -89.17 -88.61 ... 88.61 89.17 89.72
  * lon        (lon) float64 3kB 0.4167 1.25 2.083 2.917 ... 357.9 358.7 359.6
    height     float64 8B ...
  * member     (member) <U8 32B 'r1i1p1f2'
  * init       (init) int64 8B 1960
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (init, member, lead, bnds) object 60kB dask.array<chunksize=(1, 1, 1, 2), meta=np.ndarray>
    lat_bnds   (init, member, lat, bnds) float64 5kB dask.array<chunksize=(1, 1, 324, 2), meta=np.ndarray>
    lon_bnds   (init, member, lon, bnds) float64 7kB dask.array<chunksize=(1, 1, 432, 2), meta=np.ndarray>
    tas        (init, member, lead, lat, lon) float32 2GB dask.array<chunksize=(1, 1, 1, 324, 432), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            DCPP
    branch_method:          no parent
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-05-27T13:57:33Z
    ...                     ...
    variable_name:          tas
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by Met Office Hadley Ce...
    cmor_version:           3.4.0
    first_month:            1960-11-01 12:00:00
    time_axis_type:         Datetime360Day